In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pymysql
import re
from dotenv import load_dotenv
load_dotenv()
import os

In [2]:
db_connection_str = os.getenv("DB_CONNECTION_STR")

db_connection = create_engine(db_connection_str)

df = pd.read_sql('SELECT * FROM crash_db_Kentucky', con=db_connection)
df = df.replace('',np.nan)

In [3]:
df.shape

(255541, 50)

In [4]:
df.head()

,cid,createDate,createBy,dataFrom,MASTER_FILE_NUMBER,INVESTIGATING_AGENCY,LOCAL_CODE,COLLISION_STATUS_CODE,COUNTY_NAME,ROADWAY_NUMBER,...,DIRECTIONAL_ANALYSIS,MANNER_OF_COLLISION_CODE,MANNER_OF_COLLISION,ROADWAY_CHARACTER_CODE,ROADWAY_CHARACTER,LIGHT_CONDITION_CODE,LIGHT_CONDITION,RAMP_FROM_ROADWAY_ID,RAMP_TO_ROADWAY_ID,SECONDARY_COLLISION_INDICATOR
0,1000000000,2020-03-06 08:10:46,David,Kentucky,00336868,LOUISVILLE METRO POLICE DEPT,8010009447,AC,056,I 0065,...,REAR END IN TRAFFIC LANES BOTH VEHICLES MOVING,05,REAR END,None,None,None,None,None,None,None
1,1000000001,2020-03-06 08:10:46,David,Kentucky,00336869,LOUISVILLE METRO POLICE DEPT,8010009784,AC,056,None,...,ANGLE COLLISION - ONE VEHICLE TURNING LEFT,01,ANGLE,None,None,None,None,None,None,None
2,1000000002,2020-03-06 08:10:46,David,Kentucky,00336871,LOUISVILLE METRO POLICE DEPT,8010013765,AC,056,None,...,1 VEHICLE PARKED POSITION (NOT PARKING LOT/DRI...,05,REAR END,None,None,None,None,None,None,None
3,1000000003,2020-03-06 08:10:46,David,Kentucky,00336872,LOUISVILLE METRO POLICE DEPT,None,AC,056,None,...,VEHICLE BACKING,02,BACKING,None,None,None,None,None,None,None
4,1000000004,2020-03-06 08:10:46,David,Kentucky,00337553,LOUISVILLE METRO POLICE DEPT,8010007753,AC,056,None,...,SIDESWIPE-SAME DIRECTION,08,SIDESWIPE-SAME DIRECTION,None,None,None,None,None,None,None


In [5]:
df.columns

Index(['cid', 'createDate', 'createBy', 'dataFrom', 'MASTER_FILE_NUMBER',
       'INVESTIGATING_AGENCY', 'LOCAL_CODE', 'COLLISION_STATUS_CODE',
       'COUNTY_NAME', 'ROADWAY_NUMBER', 'BLOCKHOUSE_', 'ROADWAY_NAME',
       'ROADWAY_SUFFIX', 'ROADWAY_DIRECTION_CODE', 'GPS_LATITUDE_DECIMAL',
       'GPS_LONGITUDE_DECIMAL', 'MILEPOINT_DERIVED', 'COLLISION_DATE',
       'COLLISION_TIME', 'INTERSECTION_ROADWAY_', 'INTERSECTION_ROADWAY_NAME',
       'INTERSECTION_ROADWAY_SFX', 'BETWEEN_STREET_ROADWAY_1',
       'BETWEEN_STREET_ROADWAY_NAME_1', 'BETWEEN_STREET_ROADWAY_SFX_1',
       'BETWEEN_STREET_ROADWAY_2', 'BETWEEN_STREET_ROADWAY_NAME_2',
       'BETWEEN_STREET_ROADWAY_SFX_2', 'UNITS_INVOLVED',
       'MOTOR_VEHICLES_INVOLVED', 'KILLED', 'INJURED', 'WEATHER_CODE',
       'WEATHER', 'ROADWAY_CONDITION_CODE', 'ROADWAY_CONDITION',
       'HIT_RUN_INDICATOR', 'ROADWAY_TYPE_CODE', 'ROADWAY_TYPE',
       'DIRECTIONAL_ANALYSIS_CODE', 'DIRECTIONAL_ANALYSIS',
       'MANNER_OF_COLLISION_CODE', 'MAN

In [6]:
df2 = pd.DataFrame()
df2['cid'] = df['cid']
df2['city'] = df['dataFrom']
df2['crash_time'] = pd.to_datetime(df['COLLISION_DATE'] + ' ' +df['COLLISION_TIME'],errors='coerce')
df2['latitude'] = df['GPS_LATITUDE_DECIMAL']
df2['longitude'] = df['GPS_LONGITUDE_DECIMAL']
df2['injured_count'] = df['KILLED'].astype('int') + df['INJURED'].astype('int')
df2['street_name'] = df['BETWEEN_STREET_ROADWAY_1'] + df['BETWEEN_STREET_ROADWAY_NAME_1'] + df['BETWEEN_STREET_ROADWAY_SFX_1']
df2['crash_date'] = pd.to_datetime(df2['crash_time'].dt.date)

In [7]:
df2.head()

,cid,city,crash_time,latitude,longitude,injured_count,street_name,crash_date
0,1000000000,Kentucky,2010-02-08 15:07:00,38.2649,-85.7438,0,NaN,2010-02-08
1,1000000001,Kentucky,2010-02-10 13:03:00,38.252216667,-85.749,0,NaN,2010-02-10
2,1000000002,Kentucky,2010-02-26 10:37:00,38.257533333,-85.725116667,0,NaN,2010-02-26
3,1000000003,Kentucky,2010-02-28 13:15:00,38.253483333,-85.74325,0,NaN,2010-02-28
4,1000000004,Kentucky,2010-02-01 13:45:00,38.246366667,-85.7831,0,NaN,2010-02-01


In [ ]:
df2.to_sql('cleaned_crash_Kentucky',con = db_connection, if_exists="replace",index=False)

In [ ]:
df2.to_sql('home_allcleanedcrashdata',con = db_connection, if_exists="append",index=False)